# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [103]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [104]:
# 1) Seu código aqui
url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
r = requests.get(url)
r.status_code

data_json = r.json()
data_items = data_json['items']
df = pd.DataFrame(data_items)
df.head(5)



,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,57189,Município,Imbé,4310330,RS,PVL02.003581/2022-35,Encaminhado à PGFN com manifestação técnica fa...,17944.101579/2022-93,2022-06-14T18:35:50Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,1.939677e+07,0,1,29/06/2022
1,61336,Município,Lima Campos,2106003,MA,PVL02.007567/2022-19,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,5.000000e+06,0,1,29/08/2022
2,59169,Município,Butiá,4302709,RS,PVL02.005222/2022-12,Deferido,17944.102122/2022-04,2022-06-27T14:59:57Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,1.000000e+07,0,1,29/06/2022
3,42088,Município,São Bernardo do Campo,3548708,SP,PVL02.008521/2019-11,Deferido,17944.101228/2020-11,2020-03-12T14:37:43Z,Operação contratual interna,PAC 2 - Pró-Moradia,Instituição Financeira Nacional,Caixa Econômica Federal,Real,1.108261e+08,1,1,19/03/2020
4,40131,Município,Alvorada,4300604,RS,PVL02.007781/2019-61,Deferido,17944.104162/2019-87,2019-11-26T19:37:03Z,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,2.000000e+07,1,0,28/11/2019


In [105]:
# 2) Seu código aqui

top_statuses = df['data_status'].value_counts().head(3)

print("Os três status mais frequentes e suas frequências são:")
print(top_statuses)



Os três status mais frequentes e suas frequências são:
data_status
15/09/2008    22
29/08/2008    18
30/09/2021    17
Name: count, dtype: int64


In [106]:
# 3) Seu código aqui

# Ou usando slice para capturar os primeiros quatro caracteres (ano)
ano_status = df['ano_status'] = df['data_status'].str.slice(6,10)

print(ano_status)


0       2022
1       2022
2       2022
3       2020
4       2019
        ... 
4995    2015
4996    2016
4997    2016
4998    2014
4999    2016
Name: data_status, Length: 5000, dtype: object


In [107]:
# Supondo que 'ano_status' seja a coluna que contém os anos extraídos da coluna 'data_status'

frequencia_anos = df['ano_status'].value_counts()

print("Frequência de cada ano de acordo com a coluna 'ano_status':")
print(frequencia_anos)


Frequência de cada ano de acordo com a coluna 'ano_status':
ano_status
2008    451
2023    416
2022    393
2019    375
2021    329
2007    289
2010    286
2014    245
2020    235
2012    226
2011    211
2013    210
2009    208
2006    202
2018    169
2004    165
2015    122
2016    119
2017    119
2002    111
2003     79
2005     40
Name: count, dtype: int64


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [108]:
#1) Seu código aqui
import requests
import pandas as pd

def consultar_dados(uf, tipo_interessado, status):
    base_url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
    params = {'uf': uf, 'tipo_interessado': tipo_interessado}
    response = requests.get(base_url, params=params)
    
    if response.status_code == 200:
        data_json = response.json()
        data_items = data_json['items']
        df = pd.DataFrame(data_items)
        return df
    else:
        print("Não foi possível obter os dados. Código de status:", response.status_code)
        return None

# Exemplo de uso da função
uf = 'SP'  # Substitua pelo UF desejado
tipo_interessado = 'Estado'  # Pode ser 'Estado' ou 'Municipio'
status = 'Deferido'

dados_df = consultar_dados(uf, tipo_interessado,status)
if dados_df is not None:
    print(dados_df.head())  # Exibe as primeiras linhas do DataFrame retornado


   id_pleito tipo_interessado interessado  cod_ibge  uf               num_pvl  \
0       5623           Estado   São Paulo        35  SP                  None   
1      67175           Estado   São Paulo        35  SP  PVL02.003851/2023-99   
2      60069           Estado   São Paulo        35  SP  PVL02.005843/2022-04   
3      51532           Estado   São Paulo        35  SP  PVL02.006462/2021-53   
4       9030           Estado   São Paulo        35  SP                  None   

                                              status          num_processo  \
0  Encaminhado à PGFN com manifestação técnica fa...  17944.000157/2002-58   
1                                         Em análise  17944.105293/2023-68   
2                                           Deferido  17944.102355/2022-07   
3  Encaminhado à PGFN com manifestação técnica fa...  17944.104649/2021-84   
4                                           Deferido  17944.000888/2013-56   

         data_protocolo                     

In [109]:
# 2) Seu código aqui
uf = 'MG'
tipo_interessado = 'Estado'
status = 'Arquivado por decurso de prazo'

dados_df = consultar_dados(uf,tipo_interessado,status)
if dados_df is not None:
    count = len(dados_df)
    print(f"Número de solicitações para o Estado de {uf} com status '{status}': {count}")

Número de solicitações para o Estado de MG com status 'Arquivado por decurso de prazo': 40


In [110]:
# 3) Seu código aqui
# Consulta os dados para o município da Bahia
uf = 'BA'
tipo_interessado = 'Município'
status = 'Deferido'
dados_bahia_df = consultar_dados(uf,tipo_interessado, status)

if dados_bahia_df is not None:
    # Filtra as solicitações deferidas para a Bahia
    deferidas_bahia = dados_bahia_df[dados_bahia_df['status'] == 'Deferido']

    # Encontra o município com mais solicitações deferidas
    municipio_mais_deferido = deferidas_bahia['interessado'].value_counts().idxmax()
    quantidade_deferida = deferidas_bahia['interessado'].value_counts().max()

    print(f"O município da Bahia com mais solicitações deferidas é {municipio_mais_deferido} com {quantidade_deferida} solicitações deferidas.")

O município da Bahia com mais solicitações deferidas é Luís Eduardo Magalhães com 16 solicitações deferidas.


In [111]:
# 4) Seu código aqui
uf = 'BA'
estados_bahia = dados_bahia_df['uf'] == uf
estados_bahia.to_csv('arquivo_teste.csv')

In [112]:
# Consulta os dados para o Estado da Bahia
uf = 'BA'
tipo_interessado = 'Estado'
status = ''
dados_bahia_estado_df = consultar_dados(uf, tipo_interessado, status)

if dados_bahia_estado_df is not None:
    # Salvar os dados em um arquivo CSV
    dados_bahia_estado_df.to_csv('solicitacoes_bahia_estado.csv', index=False)
    print("Dados das solicitações da Bahia com interessado 'Estado' foram salvos em 'solicitacoes_bahia_estado.csv'.")
else:
    print("Não foram encontrados dados para salvar.")
    
dados_bahia_estado_df


Dados das solicitações da Bahia com interessado 'Estado' foram salvos em 'solicitacoes_bahia_estado.csv'.


,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,8858,Estado,Bahia,29,BA,None,Encaminhado à PGFN com manifestação técnica fa...,17944.000853/2014-06,2014-07-10T03:00:00Z,Operação contratual externa (com garantia da U...,Infraestrutura,Instituição Financeira Internacional,Banco Internacional para Reconstrução e Desenv...,Dólar dos EUA,4.000000e+08,1,0,16/07/2014
1,12327,Estado,Bahia,29,BA,None,Encaminhado à PGFN com manifestação técnica fa...,17944.001659/2008-91,2009-03-24T03:00:00Z,Operação contratual externa (com garantia da U...,Fortalecimento Institucional,Instituição Financeira Internacional,Banco Interamericano de Desenvolvimento,Dólar dos EUA,4.090000e+08,1,0,27/03/2009
2,19016,Estado,Bahia,29,BA,00000.000000/2009-87,Encaminhado à PGFN com manifestação técnica fa...,17944.000485/2015-79,2015-11-03T02:00:00Z,Operação contratual interna (com garantia da U...,Aditivo contratual,Instituição Financeira Nacional,Banco do Brasil S/A,Real,5.627400e+08,1,0,09/11/2015
3,18633,Estado,Bahia,29,BA,None,Deferido,19407.000073/2003-16,2004-01-30T02:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,2.150132e+08,1,0,18/03/2004
4,23723,Estado,Bahia,29,BA,PVL02.000640/2016-75,Encaminhado à PGFN com manifestação técnica fa...,17944.001273/2016-90,2017-06-06T18:33:06Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,6.000000e+08,1,1,12/06/2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,18587,Estado,Bahia,29,BA,None,Deferido,19407.000058/2002-89,2002-06-17T03:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,1.400868e+06,1,0,08/07/2002
80,45396,Estado,Bahia,29,BA,PVL02.001082/2021-22,Encaminhado à PGFN com manifestação técnica fa...,17944.102529/2021-42,2022-06-14T16:50:29Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,2.280000e+08,0,1,20/06/2022
81,18555,Estado,Bahia,29,BA,None,Deferido,19407.000050/2002-12,2002-06-11T03:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,6.774733e+05,1,0,05/07/2002
82,8336,Estado,Bahia,29,BA,None,Deferido,17944.000758/2013-13,2013-11-08T02:00:00Z,Operação contratual interna,PAC 2 - Pró-Transporte - Pavimentação e Qualif...,Instituição Financeira Nacional,Caixa Econômica Federal,Real,1.151005e+08,1,0,19/11/2013
